In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

train_data[['Cabin_Deck', 'Cabin_Num', 'Cabin_Side']] = train_data['Cabin'].str.split('/', expand=True)
test_data[['Cabin_Deck', 'Cabin_Num', 'Cabin_Side']] = test_data['Cabin'].str.split('/', expand=True)

# Separate target
y = train_data["Transported"]
X = train_data.drop(columns=["Transported", "PassengerId"])  # Remove target + ID
test_ids = test_data["PassengerId"]
X_test_raw = test_data.drop(columns=["PassengerId"])



train_data['Cabin_Num'] = pd.to_numeric(train_data['Cabin_Num'], errors='coerce')
test_data['Cabin_Num'] = pd.to_numeric(test_data['Cabin_Num'], errors='coerce')





num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include='object').columns.tolist()

if 'Cabin_Num' not in num_features:
    num_features.append('Cabin_Num')
for col in ['Cabin_Deck', 'Cabin_Side']:
    if col not in cat_features:
        cat_features.append(col)


numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, num_features),
    ("cat", categorical_pipeline, cat_features)
])


X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test_raw)


X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=70)


model = XGBClassifier(
    learning_rate=0.05,
    max_depth=5,
    n_estimators=300,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)


y_val_pred = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))


y_test_pred = model.predict(X_test_processed)


submission = pd.DataFrame({
    'PassengerId': test_ids,
    'Transported': y_test_pred.astype(bool)
})
submission.to_csv("submission.csv", index=False)
print("Submission file created!")

Accuracy: 0.8263369752731455
              precision    recall  f1-score   support

       False       0.83      0.82      0.82       867
        True       0.82      0.83      0.83       872

    accuracy                           0.83      1739
   macro avg       0.83      0.83      0.83      1739
weighted avg       0.83      0.83      0.83      1739

Submission file created!
